In [ ]:
!pip install rich==6.2.0
!pip install cobra
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.util.solver import linear_reaction_coefficients
!pip install gurobipy
import gurobipy as gp
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import single_reaction_deletion
from cobra.flux_analysis.deletion import double_reaction_deletion
import os

In [ ]:
! git clone https://github.com/SysBioChalmers/yeast-GEM.git

In [ ]:
!pip install -r yeast-GEM/requirements/requirements.txt  # installs all dependencies               # creates a .env file for locating the root

In [5]:
os.chdir('yeast-GEM')
! touch .env
from dotenv import find_dotenv
from os.path import dirname

# find .env + define paths:
dotenv_path = find_dotenv()
REPO_PATH = dirname(dotenv_path)
MODEL_PATH = f"{REPO_PATH}/model/yeast-GEM.xml"

In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location("alo", "/content/yeast-GEM/code/io.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
model = foo.read_yeast_model()

In [7]:
model.solver = "glpk"

In [ ]:
model.metabolites.get_by_id("s_0681[e]") #s_0681[e] is the id of ethanol in the model

In [ ]:
model.objective = "r_1761"
model.summary()

In [ ]:
flux_variability_analysis(model,fraction_of_optimum=0.95)
model.summary(fva=0.95)

In [ ]:
df = single_reaction_deletion(model)
df

In [ ]:
df.max(numeric_only=True)

In [13]:
def filter_unwanted_reactions_from_model(model):
  reactions_to_remove = [] #Clean list
  for ele in range(0,len(model.reactions)):
    if len(model.reactions[ele].compartments) > 1:
      reactions_to_remove.append(model.reactions[ele].id)
    elif model.reactions[ele] in model.exchanges:
      reactions_to_remove.append(model.reactions[ele].id)
    else:
      model2 = model.copy()
      model2.reactions.get_by_id(model.reactions[ele].id).bounds = (0,0)
      sol_tmp = model2.slim_optimize()
      if sol_tmp < 1e-4:
        reactions_to_remove.append(model.reactions[ele].id)
  model3 = model.copy()
  model3.remove_reactions(reactions_to_remove) #Remove those reactions
  return model3

In [ ]:
model_copy = filter_unwanted_reactions_from_model(model) #Running this took up all the RAM and didn't finish running on my laptop

In [ ]:
double_reaction_deletion(model)